# Notebook này chứa các đoạn code sử dụng để xử lý hàng loạt trên dữ liệu. Nếu muốn hiểu thêm chi tiết về đồ án, vui lòng chuyển đến Final.ipynb

# Đồ án cuối kỳ - Phân tích chủ đề văn bản

Nhóm: 12

Thành viên nhóm:
- Vũ Đăng Hoàng Long - MSSV: 18120203
- Nguyễn Huỳnh Đại Lợi - MSSV: 18120198

---
# Giới thiệu đồ án

Chủ đề: Nhận diện chủ đề của một đoạn văn bản bất kỳ.

Input: một đoạn văn bản bất kỳ.

Output: một trong 18 phân lớp sau:

| | | |
| :- | :- | :- |
| 1. thời sự quốc tế | 2. thời sự trong nước | 3. du lịch |
| 4. kinh doanh | 5. giải trí | 6. công nghệ |
| 7. nhà đất | 8. sức khỏe | 9. giáo dục |
| 10. khoa học | 11. thể thao | 12. văn hóa |
| 13. pháp luật | 14. yêu | 15. xe |
| 16. thời trang | 17. nhịp sống trẻ | 18. ăn gì |

Nguồn dữ liệu: tất cả bài báo thu thập từ trang báo điện tử Tuổi trẻ Online (https://tuoitre.vn/).

Mục đích:
- Khách quan: phục vụ việc nhận diện chủ đề một cách tự động.
- Chủ quan: lọc các bài viết trên mạng xã hội theo chủ đề mà em quan tâm để tránh lãng phí thời gian lướt facebook chỉ để tìm chủ đề mà em quan tâm 🥴.

---
# Phần đồ án (xử lý hàng loạt toàn dữ liệu)

## Cài đặt thư viện

(có thể bỏ qua cell ở đây nếu cài rồi)

In [1]:
%%capture
# Cài đặt thư viện cần thiết
!pip3 install pandas;
!pip3 install tqdm;
!pip3 install bs4;
!pip3 install regex;
!pip3 install numpy;
!pip3 install pyvi;
!pip3 install gensim;
!pip3 install matplotlib;
!pip3 install seaborn;
!pip3 install -U scikit-learn;
!pip3 install lime;

## Import thư viện

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 125

import seaborn as sns

import pandas as pd
import numpy as np
import regex as re
import time # Dùng để sleep chương trình
from tqdm.notebook import tqdm # Hiện thanh progress cho đẹp :D
tqdm.pandas()

# Thư viện để request và parse HTML
import requests
from bs4 import BeautifulSoup

# Các thư viện liên quan tới ngôn ngữ và NLP
from pyvi import ViTokenizer # Thư viện NLP tiếng Việt
import gensim
import unicodedata # Thư viện unicode

# Trực quan hóa văn bản
from lime import lime_text

# Dùng để lưu lại model
import pickle

# Thư viện liên quan của Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import feature_selection

# Tạo pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer, make_column_selector

# Các mô hình học
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier # Phương pháp bagging

c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


---
## 1. Thu thập dữ liệu

In [3]:
# Thiết lập đường dẫn cho phần 1
dir_1 = "src/scraped_data/"

In [6]:
---
## 1. Thu thập dữ liệu

# Thiết lập đường dẫn cho phần 1
dir_1 = "src/scraped_data/"def single_request_scraping(index = 1, limit_retry = 100, sleep_time = 0.05):
    '''
    Thu thập các trang tin trong mục tin mới nhất của báo Tuổi Trẻ.
    Mỗi lần thu thập 20 tin. Sử dụng số index để thu thập các trang liên tiếp.
    Với index = 1 tức lấy 20 trang tin mới nhất.

    Lưu ý:
        Khi lấy liên tục cần chọn thời điểm
        vì có khả năng báo cập nhật tin tức mới sẽ làm 
        tin ở trang trước bị đẩy xuống trang sau.

    Param:
        index: số chỉ trang cần request.
    '''
    allow_status = [200, 301]
    s = requests.Session()
    s.headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'
    url = f"https://tuoitre.vn/timeline/0/trang-{index}.htm"

    # Lấy danh sách news_items
    try:
        response = s.get(url)
        try_left = limit_retry
        while (response.status_code not in allow_status and try_left > 0):
            print(f"Loi {response.status_code} tai trang {index}")
            response = s.get(url)
            try_left -= 1
        html_text = response.text
    except:
        print(f"Loi request tai trang {index}")
        return None
    
    html_tree = BeautifulSoup(html_text, 'html.parser')
    news_items = html_tree.findAll('li', class_='news-item')
    
    # Kiểm tra lỗi nếu không lấy được bất cứ item nào
    if (len(news_items) == 0):
        print(f'Trang {index} khong lay duoc item.')
        print(html_text)
        return None
    elif (len(news_items) != 20):
        print(f"Warning: Trang {index} chi lay duoc {len(news_items)} items.")
    
    # Lấy ra link, title và category từ news_items
    raw_data = pd.DataFrame(columns=["links","title","description","content","class"])
    for item in news_items:
        try:
            title = item.find('h3', class_='title-news').text.replace('\n','')
        except:
            title = ""
        link = "https://tuoitre.vn" + item.find('a').attrs["href"]
        try:
            category = item.find('a', class_='category-name').text
        except:
            category = ""
        raw_data = raw_data.append({"links":link, "title":title, "class":category}, ignore_index=True)
    
    # Tiến hành lấy nội dung từng link
    for _, row in raw_data.iterrows():
        try:
            response = s.get(row["links"])
            try_left = limit_retry
            while (response.status_code not in allow_status and try_left > 0):
                print(f"Loi {response.status_code} tai link {row['links']}")
                response = s.get(row["links"])
                try_left -= 1
            news_page = response.content
        except:
            print(f"Loi request tai link {row['links']}")
            row["description"] = ""
            row["content"] = ""
            continue
            
        news_tree = BeautifulSoup(news_page, "html.parser")
        # Lấy mô tả
        try:
            row["description"] = news_tree.find("h2", class_="sapo").text
        except:
            row["description"] = ''
        # Lấy nội dung
        try:
            body = news_tree.find("div", id="main-detail-body")
            content = body.findChildren("p", recursive=False)
            row["content"] = ""
            for x in content:
                row["content"] += x.text
        except:
            row["content"] = ''
        time.sleep(sleep_time)
    
    return raw_data

In [7]:
#Batch scraping
def batch_scraping(num = 200, output_dir = ""):
    '''
    Thực hiện thu thập toàn bộ dữ liệu trên trang tuoitre.vn
    
    Param:
        num: số lượng trang request cho 1 batch (1 trang = 20 bài báo). Mặc định 4000 bài báo.
    '''
    iter_num = 0  # Số batch bắt đầu
    continue_flag = True # Cờ hiệu kết thúc vòng lặp khi xảy ra lỗi

    while (continue_flag):
        
        '''
        Khởi tạo dataframe rỗng. 
        Sau đó lấy đủ 1 số trang cho 1 batch.
        Rồi export file csv.
        '''
        batch_df = pd.DataFrame(columns=["links","title","description","content","class"])
        for index in range(iter_num*num+1,(iter_num+1)*num+1):
            data = single_request_scraping(index)
            if (data is None):
                continue_flag = False
                break
            print(f"Page {index} complete!")
            batch_df = batch_df.append(data)
        batch_df.to_csv(output_dir + f'crawling_{iter_num}.csv',index=False,encoding="utf-8")    
        iter_num+=1

In [8]:
#batch_scraping(output_dir = dir_1)

---
## 3. Tiền xử lý bước đầu

In [19]:
def batch_prepreprocess():
    total_files = 218
    text_attrs = ["title","description","content","class"]
    prev_df = None
    
    for index in tqdm(range(total_files)):
        df=pd.read_csv(f'src/scraped_data/crawling_{index}.csv')
        # Xóa dòng thiếu
        df.dropna(axis=0, how="any", inplace=True)
        # Xóa dòng trùng
        df.drop_duplicates(inplace=True)
        # Xóa dòng trùng với file trước
        if (prev_df is not None):
            df = df.merge(prev_df, how='outer', indicator=True).loc[lambda x : x['_merge'] == 'left_only']
            df.drop(['_merge'],axis=1,inplace=True)
        # Chuẩn hóa unicode và dấu
        for attr in text_attrs:
            df[attr]=df[attr].apply(covert_unicode)
            df[attr]=df[attr].str.lower()
            df[attr]=df[attr].apply(chuan_hoa_dau_cau_tieng_viet)
        df.to_csv(f'src/scraped_data/crawling_{index}.csv', index=False)
        prev_df = df
        
    print("done.")

In [20]:
#batch_prepreprocess()

## 5. Tiền xử lý sau cùng toàn dữ liệu